In [1]:
!pip install PyGithub
!pip install tqdm

In [2]:
with open("token.txt","r") as fil:
    TOKEN=fil.readline()

In [3]:
import os
import os.path

if os.path.exists("repos.txt"):
    
    with open("repos.txt","r") as fil:
        repos=fil.readlines()

    repos = [x.replace("\n","") for x in repos]

else:
    print("No repo list exists, aborting")
    raise NotImplementedError

In [4]:
import os
import os.path

folder = "repos"

if folder:
    os.chdir(".")

    if os.path.isdir(folder):
        print("Folder already exists")
    else:
        print("Creating folder",folder)
        os.mkdir(folder)


Folder already exists


In [5]:
if os.path.exists("last_worked.txt"):
    
    with open("last_worked.txt","r") as fil:
        last_worked_on=fil.readline()
    position = repos.index(last_worked_on)
    
    last_filename = last_worked_on.replace("/","-")+".csv"
    try:
        os.remove(folder+"/"+last_filename)
    except:
        print("File not found, so not removing it")
    repos = repos[position:]


File not found, so not removing it


In [6]:
from github import Github

g = Github(TOKEN)


In [7]:
import pandas as pd
import tqdm
from github import GithubException


def get_commits_by_repo_name(repo_name):

    with open("last_worked.txt","w") as fil:
        fil.write(repo_name)
    
    authors = []
    dates = []
    empties_count = 0

    repo = g.get_repo(repo_name)
    
    commits = repo.get_commits()
    try:
        
        for commit in tqdm.tqdm(commits, desc="Commits progress", position=1, leave=True):
            authors.append(commit.commit.raw_data["author"]["name"])
            dates.append(commit.commit.raw_data["author"]["date"])
    except GithubException as err:
        if err.data["message"] == "Git Repository is empty.":
            print("Found an empty repo.")
            empties_count+=1
        else:
            raise err
    return authors, dates, empties_count

def create_and_save_dataframe(authors, dates, repo_name, folder=""):
    if authors:
        df = pd.DataFrame({"commit_author":authors, "commit_date": dates})
        if folder:
            folder=folder+"/"
        df.to_csv(folder+repo_name.replace("/","-")+".csv")
    return
    


In [8]:
for repo_name in tqdm.tqdm(list(repos), desc="Repos progress"):
    authors, dates, empties_count = get_commits_by_repo_name(repo_name)
    create_and_save_dataframe(authors, dates, repo_name, folder)

print("Total nuber of empty repos:", empties_count)


Repos progress:   0%|                                             | 0/1011 [00:00<?, ?it/s]
Commits progress: 0it [00:00, ?it/s]
Commits progress: 1it [00:00,  1.73it/s]
Commits progress: 2it [00:00,  2.58it/s]
Commits progress: 3it [00:01,  2.97it/s]
Commits progress: 4it [00:01,  3.28it/s]
Commits progress: 5it [00:01,  3.49it/s]
Commits progress: 6it [00:01,  3.66it/s]
Commits progress: 7it [00:02,  3.79it/s]
Commits progress: 8it [00:02,  3.56it/s]
Commits progress: 9it [00:02,  3.38it/s]
Commits progress: 10it [00:03,  3.53it/s]
Commits progress: 11it [00:03,  3.55it/s]
Commits progress: 12it [00:03,  3.43it/s]
Commits progress: 13it [00:03,  3.53it/s]
Commits progress: 14it [00:04,  3.48it/s]
Commits progress: 15it [00:04,  3.28it/s]
Commits progress: 16it [00:04,  3.18it/s]
Commits progress: 17it [00:05,  3.33it/s]
Commits progress: 18it [00:05,  3.31it/s]
Commits progress: 19it [00:05,  3.32it/s]
Commits progress: 20it [00:05,  3.46it/s]
Commits progress: 21it [00:06,  3.23it/s

Found an empty repo.



Commits progress: 0it [00:00, ?it/s]
Commits progress: 1it [00:00,  1.61it/s]
Commits progress: 2it [00:00,  2.49it/s]
Commits progress: 3it [00:01,  2.86it/s]
Commits progress: 4it [00:01,  3.11it/s]
Commits progress: 5it [00:01,  3.46it/s]
Commits progress: 6it [00:01,  3.54it/s]
Commits progress: 7it [00:02,  3.48it/s]
Commits progress: 8it [00:02,  3.58it/s]
Commits progress: 9it [00:02,  3.70it/s]
Commits progress: 10it [00:02,  3.84it/s]
Commits progress: 11it [00:05,  1.11it/s]
Commits progress: 12it [00:05,  1.43it/s]
Commits progress: 13it [00:05,  1.77it/s]
Commits progress: 14it [00:06,  2.12it/s]
Commits progress: 15it [00:06,  2.45it/s]
Commits progress: 16it [00:06,  2.72it/s]
Commits progress: 17it [00:06,  3.05it/s]
Commits progress: 18it [00:07,  3.20it/s]
Commits progress: 19it [00:07,  3.43it/s]
Commits progress: 20it [00:07,  3.53it/s]
Commits progress: 21it [00:07,  3.65it/s]
Commits progress: 22it [00:08,  3.62it/s]
Commits progress: 23it [00:08,  3.62it/s]
Commi

Found an empty repo.



Commits progress: 0it [00:00, ?it/s]
Commits progress: 1it [00:00,  1.57it/s]
Commits progress: 2it [00:00,  2.09it/s]
Repos progress:   1%|▍                                   | 12/1011 [01:03<36:02,  2.16s/it]
Commits progress: 0it [00:00, ?it/s]
Commits progress: 1it [00:00,  1.44it/s]
Commits progress: 2it [00:01,  2.05it/s]
Commits progress: 3it [00:01,  2.53it/s]
Commits progress: 4it [00:01,  2.82it/s]
Commits progress: 5it [00:01,  3.09it/s]
Commits progress: 6it [00:02,  3.25it/s]
Commits progress: 7it [00:02,  3.29it/s]
Commits progress: 8it [00:02,  3.24it/s]
Commits progress: 9it [00:03,  3.21it/s]
Commits progress: 10it [00:03,  3.39it/s]
Commits progress: 11it [00:03,  3.56it/s]
Commits progress: 12it [00:03,  3.68it/s]
Commits progress: 13it [00:04,  3.78it/s]
Commits progress: 14it [00:04,  3.61it/s]
Commits progress: 15it [00:04,  3.49it/s]
Commits progress: 16it [00:04,  3.53it/s]
Commits progress: 17it [00:05,  3.41it/s]
Commits progress: 18it [00:05,  3.41it/s]
Comm

...

Commits progress: 1152it [06:42,  3.85it/s]
Commits progress: 1153it [06:42,  3.77it/s]
Commits progress: 1154it [06:42,  3.83it/s]
Commits progress: 1155it [06:43,  2.73it/s]
Commits progress: 1156it [06:43,  2.96it/s]
Commits progress: 1157it [06:44,  2.43it/s]
Commits progress: 1158it [06:44,  2.76it/s]
Commits progress: 1159it [06:45,  2.88it/s]
Commits progress: 1160it [06:45,  3.15it/s]
Commits progress: 1161it [06:45,  3.42it/s]
Commits progress: 1162it [06:45,  3.60it/s]
Commits progress: 1163it [06:46,  3.63it/s]
Commits progress: 1164it [06:46,  3.69it/s]
Commits progress: 1165it [06:46,  3.93it/s]
Commits progress: 1166it [06:46,  4.18it/s]
Commits progress: 1167it [06:46,  4.17it/s]
Commits progress: 1168it [06:47,  4.11it/s]
Commits progress: 1169it [06:47,  3.95it/s]
Commits progress: 1170it [06:47,  3.62it/s]
Commits progress: 1171it [06:48,  1.86it/s]
Commits progress: 1172it [06:50,  1.33it/s]
Commits progress: 1173it [06:50,  1.68it/s]
Commits progress: 1174it [06:50,

Commits progress: 18it [00:05,  3.50it/s]
Commits progress: 19it [00:06,  3.34it/s]
Commits progress: 20it [00:06,  3.57it/s]
Commits progress: 21it [00:06,  3.78it/s]
Commits progress: 22it [00:07,  2.49it/s]
Commits progress: 23it [00:07,  2.81it/s]
Commits progress: 24it [00:07,  3.14it/s]
Commits progress: 25it [00:08,  3.42it/s]
Commits progress: 26it [00:08,  3.69it/s]
Commits progress: 27it [00:08,  3.92it/s]
Commits progress: 28it [00:08,  4.00it/s]
Commits progress: 29it [00:08,  4.22it/s]
Commits progress: 30it [00:09,  4.25it/s]
Commits progress: 31it [00:10,  2.36it/s]
Commits progress: 32it [00:10,  2.66it/s]
Commits progress: 33it [00:10,  3.05it/s]
Commits progress: 34it [00:10,  3.28it/s]
Commits progress: 35it [00:11,  3.67it/s]
Commits progress: 36it [00:11,  3.81it/s]
Commits progress: 37it [00:11,  3.89it/s]
Commits progress: 38it [00:11,  3.93it/s]
Commits progress: 39it [00:11,  3.97it/s]
Commits progress: 40it [00:12,  4.19it/s]
Commits progress: 41it [00:12,  4.

Commits progress: 57it [00:17,  2.48it/s]
Commits progress: 58it [00:17,  2.55it/s]
Commits progress: 59it [00:18,  2.77it/s]
Commits progress: 60it [00:18,  2.22it/s]
Commits progress: 61it [00:20,  1.45it/s]
Commits progress: 62it [00:20,  1.77it/s]
Commits progress: 63it [00:20,  2.05it/s]
Commits progress: 64it [00:20,  2.40it/s]
Commits progress: 65it [00:21,  2.09it/s]
Commits progress: 66it [00:21,  2.37it/s]
Commits progress: 67it [00:22,  2.49it/s]
Commits progress: 68it [00:22,  2.72it/s]
Commits progress: 69it [00:22,  3.04it/s]
Commits progress: 70it [00:23,  3.29it/s]
Commits progress: 71it [00:23,  3.50it/s]
Commits progress: 72it [00:23,  3.61it/s]
Commits progress: 73it [00:23,  3.61it/s]
Commits progress: 74it [00:24,  3.77it/s]
Commits progress: 75it [00:24,  3.64it/s]
Commits progress: 76it [00:24,  3.66it/s]
Commits progress: 77it [00:24,  3.64it/s]
Commits progress: 78it [00:25,  3.76it/s]
Commits progress: 79it [00:25,  3.80it/s]
Commits progress: 80it [00:25,  3.

Commits progress: 236it [01:11,  3.26it/s]
Commits progress: 237it [01:11,  3.53it/s]
Commits progress: 238it [01:11,  3.82it/s]
Commits progress: 239it [01:12,  4.04it/s]
Commits progress: 240it [01:12,  3.76it/s]
Commits progress: 241it [01:13,  2.30it/s]
Commits progress: 242it [01:13,  2.70it/s]
Commits progress: 243it [01:13,  3.03it/s]
Commits progress: 244it [01:14,  3.25it/s]
Commits progress: 245it [01:14,  3.27it/s]
Commits progress: 246it [01:14,  3.46it/s]
Commits progress: 247it [01:14,  3.53it/s]
Commits progress: 248it [01:15,  3.60it/s]
Commits progress: 249it [01:15,  3.73it/s]
Commits progress: 250it [01:15,  3.87it/s]
Commits progress: 251it [01:16,  2.55it/s]
Commits progress: 252it [01:16,  2.85it/s]
Commits progress: 253it [01:16,  3.10it/s]
Commits progress: 254it [01:17,  3.33it/s]
Commits progress: 255it [01:17,  3.53it/s]
Commits progress: 256it [01:17,  3.69it/s]
Commits progress: 257it [01:17,  3.86it/s]
Commits progress: 258it [01:18,  3.82it/s]
Commits pro

RateLimitExceededException: 403 {"message": "API rate limit exceeded for user ID 11365719.", "documentation_url": "https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting"}